## Single tournament

In [15]:
'''Core updated as of March 7th  with Leonardo's fixes'''
import numpy as np
from numpy import random as npr
import pandas as pd
from matplotlib import pyplot as plt
import copy
import itertools
import copy

In [11]:
# INITIAL DATA:_____________________________________________
cooperate = np.array( [ 1, 0 ] )
defect = np.array( [ 0, 1 ] )

# default Payoff matrix
default_R = 2
default_S = 0
default_T = 3
default_P = 1
Payoff = np.array( [ [ default_R , default_S ], [ default_T, default_P ] ] )

N_ROUNDS = 10

In [9]:
def tourney(participants={},N_rounds = 10, M = Payoff):
    # Retrieve only the names of the strategies repeated as many times as their frequency allows.
    player_list_from_dict = []
    for item in (list(participants.keys())):
        for i in range (participants[item]):
            player_list_from_dict.append(item)
    # Compute the combinations; List which will be used to have the scores of each match by the side of the two opponents
    list_with_scores = list(itertools.combinations(player_list_from_dict, 2))
    N_giornata = len(list_with_scores)
    # Loop over the combinations one by one
    for i in range (N_giornata):
        # Player1 and Player2 strategies
        player1 = list_with_scores[i][0]
        player2 = list_with_scores[i][1]
        # Initiliaze an empty list to store the results over the match
        AllRewards = []
        # Do the match between the two players and update the rewards; The ordered syntax for match is:
        # match( key_1, key_2, N_rounds = 10, M = Payoff ) : 
        AllRewards =  match(player1, player2, N_rounds, M)
        # Retrieve the final scores for Player1 and Player2 as the last element in the list
        R1, R2 = AllRewards[-1][0], AllRewards[-1][1]
        list_with_scores[i] += (R1,R2)
    # Un-comment the following print to debug
    #for item in list_with_scores: print(item)
    
    # Good, now each combination has the corresponding score: we now need to assign it to the respective players and calculate the final score
    strat = list(participants.keys())
    freq = [participants[key] for key in strat]
    named =[]
    # Add identification to the layer trough the use of a numerical value from 1 up to their frequency BUT PRESERVE THE ORDER
    for key,f in zip (strat,freq):
        for j in range(1,f+1):
            if (f!=0):
                named.append(key + str(j))
    # Compute the permutations preserving the order
    with_names = list(itertools.combinations(named,2))
    # Copy the scores for each match over to the ones with identification
    # In list_with_scores[I][J] we have: I = ordered number of the combination, and J:
    # J=0 is 1st strategy; J=1 is 2nd strategy; J=2 is Player1 score; J=3 is Player2 score
    for i in range (N_giornata): with_names[i] += (list_with_scores[i][2],list_with_scores[i][3] )
    # Un-comment the following print to debug
    #for item in with_names: print(item)

    # Compute the score for each named player and add it to the list
    FinalScores = []
    for player in named:
        SumOver1 = np.sum([ with_names[i][2] for i in range (N_giornata) if (with_names[i][0] == player) ])
        SumOver2 = np.sum([ with_names[i][3] for i in range (N_giornata) if (with_names[i][1] == player) ])
        #FinalScores.append(SumOver1 + SumOver2)
        # Normalize the score by the number of opponents in the tourney
        FinalScores.append( (SumOver1 + SumOver2)/(np.sum(freq)-1) )
    Results = named
    
    for i in range (len(Results)):
        Results[i] = (Results[i],FinalScores[i])

    return ( Results )    

In [12]:
# Tourney test
test_part ={ 
    'NiceGuy' : 1,
    'BadGuy' : 1,
    'MainlyNice' : 1,
    'MainlyBad' : 1,
    'RandomGuy' : 1,
    'TitForTat' : 1,
    'ResentfulGuy' : 1,
    'TrustingGuy' : 1,
    'Thanos' : 1,
    'MidResentful' : 1,
    'ReverseTft': 1,
    'ScammingGuy': 1,
}

result = tourney(test_part,N_rounds = N_ROUNDS, M = Payoff)
#for item in result: print(item)
print("Here's the Ranking for the top 12 players at the end of the tourney:")
# Sorted gives a copy, while using LIST.sort() does it in place.
Ranking = sorted(result,reverse=True, key=lambda x: x[1])
for item in (Ranking[:12]): print(f"{item[0]:<15} {item[1]:.3f}")

Here's the Ranking for the top 12 players at the end of the tourney:
BadGuy1         17.455
MainlyBad1      16.091
TitForTat1      15.909
MidResentful1   15.909
RandomGuy1      15.818
TrustingGuy1    15.727
ScammingGuy1    15.727
MainlyNice1     15.091
ResentfulGuy1   15.000
Thanos1         14.091
ReverseTft1     12.727
NiceGuy1        10.909


In [21]:
# The iterated tournament function is slightly different; Inside the function the tournament is repeated thanks to a 
# while loop and the result is a list of lists of tuples with the final scores of each tournament

#Try to iterate the tournament process: new parameter "r" to set the number of iterations of the tournament
def r_tournament(participants={},N_rounds = 10, r=50, M = Payoff):

    #create a copy of the participants dictionary in order to not change the original one
    P = copy.copy(participants)
    #set a counter for the while loop
    t_counter = 0 
    #array to store all the results of each tournamen
    classification = [] 
    # create a data frame to memorize the strategies at the start of each tournament
    data_frames = []
    time_evolution_0 = pd.DataFrame(list(P.items()),columns = ['Strategies','N'])
    time_evolution_0['tournament'] = t_counter 
    data_frames.append(time_evolution_0)

    # start the while loop in order to iterate the tournament. The number of iterations is set at the start
    while t_counter < r:
        
        # Call the function to play the tournament and append the results
        result = tourney(P,N_rounds = N_ROUNDS, M = Payoff)
        ranking = sorted(result,reverse=True, key=lambda x: x[1])
        classification.append(ranking)
        
        #Update the participants dictionary: remove the last classified and add one more player with the winning strategy
        looser = classification[t_counter][-1][0] 
        winner = classification[t_counter][0][0]
        looser_points = classification[t_counter][-1][1] 
        winner_points = classification[t_counter][0][1] 
        #print("Winner: ",winner, winner_points, "Looser: ", looser, looser_points)
        if (np.isclose(winner_points, looser_points)): break
        
        #retrieve the strategy name to eliminate/increase in the dictionary removing the numerical characters
        l_strategy = ''.join([char for char in looser if char.isalpha()])
        w_strategy = ''.join([char for char in winner if char.isalpha()])
        
        #increase/decrease the presence of the winner/looser strategies
        P[l_strategy] -= 1
        P[w_strategy] += 1
        
        #test
        #time_evolution['tournament'] += t_counter

        #update the counter 
        t_counter += 1 

        #update the data frame for the time evolution with a new column
        #new_line = [ s[1] for s in list(participants.items())]
        #time_evolution.loc[len(time_evolution)] = new_line
        time_evolution_i = pd.DataFrame(list(P.items()),columns = ['Strategies','N'])
        time_evolution_i['tournament'] = t_counter 
        data_frames.append(time_evolution_i)

    data_frame_tot = pd.concat(data_frames, axis=0)
    
    return classification, data_frame_tot  

In [22]:
# Iterated tournament function test
test_part = {
    'NiceGuy' : 3,
    'BadGuy' : 1,
    'MainlyNice' : 5,
    'MainlyBad' : 2,
    'TitForTat' : 5,
    'RandomGuy' : 5,
    'ResentfulGuy' : 4,
    'TrustingGuy' : 1,
    'Thanos' : 2,
    'MidResentful': 3,
    'ReverseTft': 6,
    'ScammingGuy': 4
}

x,y = r_tournament(test_part,r = 300,N_rounds = R, M = Payoff)


In [20]:
print(x)
print(y)

[[('BadGuy1', 18.35), ('ResentfulGuy4', 17.875), ('MidResentful3', 17.6), ('ResentfulGuy1', 17.55), ('ResentfulGuy3', 17.525), ('MidResentful1', 17.525), ('MidResentful2', 17.425), ('ResentfulGuy2', 17.4), ('ScammingGuy3', 17.075), ('ScammingGuy2', 16.925), ('ScammingGuy1', 16.775), ('RandomGuy3', 16.725), ('TitForTat3', 16.6), ('TitForTat4', 16.6), ('RandomGuy5', 16.5), ('MainlyBad2', 16.475), ('TitForTat5', 16.4), ('TitForTat1', 16.375), ('TitForTat2', 16.25), ('MainlyBad1', 16.2), ('RandomGuy1', 16.175), ('ScammingGuy4', 15.825), ('TrustingGuy1', 15.675), ('RandomGuy2', 15.3), ('RandomGuy4', 15.15), ('ReverseTft5', 15.15), ('Thanos1', 15.025), ('ReverseTft6', 15.0), ('Thanos2', 14.95), ('ReverseTft3', 14.675), ('ReverseTft4', 14.475), ('ReverseTft1', 14.375), ('ReverseTft2', 14.225), ('MainlyNice5', 14.125), ('MainlyNice2', 14.05), ('MainlyNice3', 13.85), ('MainlyNice1', 13.575), ('MainlyNice4', 13.3), ('NiceGuy1', 12.2), ('NiceGuy3', 12.15), ('NiceGuy2', 12.0)], [('BadGuy2', 18.5),

In [16]:
# Iterated tournament function test
test_part = {
    'NiceGuy' : 3,
    'BadGuy' : 1,
    'MainlyNice' : 5,
    'MainlyBad' : 2,
    'TitForTat' : 5,
    'RandomGuy' : 5,
    'ResentfulGuy' : 4,
    'TrustingGuy' : 1,
    'Thanos' : 2,
    'MidResentful': 3,
    'ReverseTft': 6,
    'ScammingGuy': 4
}

diff_rounds_res = []
diff_rounds_te = []
#total_data = []

for R in range(3,5):
    x,y = r_tournament(test_part,r = 300,N_rounds = R, M = Payoff)
    diff_rounds_res.append(x)
    diff_rounds_te.append(y)

TE = pd.concat(diff_rounds_te, axis=1)   

InvalidIndexError: Reindexing only valid with uniquely valued Index objects